**Name : Bhargav Pandya** 

**Student Id: 202218055**

**Assignment - 07**

In [1]:
import numpy as np

In [3]:
A = [[ 3, -4,  2],
     [ 1, -7, -3],
     [-2,  4,  7]]

In [4]:
A = np.array(A)

In [5]:
print("Payoff Matrix A :\n", A)

Payoff Matrix A :
 [[ 3 -4  2]
 [ 1 -7 -3]
 [-2  4  7]]


formulate the game as a linear programming problem

In [6]:
def convert_LPP(A):
    k = abs(A.flatten().min()) + 1 #we add k = minimum numberof payoff +1 , for value of game > 0
    A1 = A + k
    B = A1.T
 
    # Formulation for Max u
    print('-' * 40)
    print('Max u')
    print('Subject to')
    for i in range(B.shape[0]):
        row = ''
        for j in range(len(B[0])):
            row = row + '({})x{} + '.format(B[i][j], j+1)
        row = row[:-2]
        row = row + '>= u'
        print(row)
    row1 = ''
    for k in range(len(B[0])):
        row1 = row1 + 'x{} + '.format(k+1)
    row1 = row1[:-2]
    row1 = row1 + '= 1'
    print(row1)
    print('xi >= 0')
    print('-' * 40)

    # Formulation for min v
    print('Min v')
    print('Subject to')
    for i in range(A1.shape[0]):
        row = ''
        for j in range(len(A1[0])):
            row = row + '({})y{} + '.format(A1[i][j], j+1)
        row = row[:-2]
        row = row + '<= v'
        print(row)
    row1 = ''
    for k in range(len(A1[0])):
        row1 = row1 + 'y{} + '.format(k+1)
    row1 = row1[:-2]
    row1 = row1 + '= 1'
    print(row1)
    print('yi >= 0')
    print('-' * 40)

In [7]:
convert_LPP(A)

----------------------------------------
Max u
Subject to
(11)x1 + (9)x2 + (6)x3 >= u
(4)x1 + (1)x2 + (12)x3 >= u
(10)x1 + (5)x2 + (15)x3 >= u
x1 + x2 + x3 = 1
xi >= 0
----------------------------------------
Min v
Subject to
(11)y1 + (4)y2 + (10)y3 <= v
(9)y1 + (1)y2 + (5)y3 <= v
(6)y1 + (12)y2 + (15)y3 <= v
y1 + y2 + y3 = 1
yi >= 0
----------------------------------------


Solve a linear programming problem using Python.

In [8]:
#Function for calculating zj
def Zj(A, t, b):
  s = 0
  for j in range(len(b)):
    s = s + b[j]*A[j][t]
  return s

In [12]:
n,m = A.shape

In [13]:
#Function for finding minimum
def Min(array, m_type = ' '):
  k = 0
  if m_type == 'zj_cj':
    min = 0
    for i in range(len(array)):
      if array[i] < min:
        min = array[i]
        k = i
    return min, k

  else:
    min = 987654321
    for i in range(len(array)):
      if array[i] < min and array[i] > 0:
        min = array[i]
        k = i
    return min, k

In [14]:
#Function to implements LPP method to solve mxn game 
def Value_of_game(A,n,m):
  
  k = abs(A.flatten().min()) + 1 #we add k = minimum numberof payoff +1 , for value of game > 0
  A = A + k
  I = np.identity(n)
  A = np.column_stack((A, I))
  A = A.tolist()

  x = [1, 1, 1] # Solution matrix
  b = [0] * n  #Cost of basic variable
  c = ([1] * m) + b  #Cost of the objective  

  var = ['B'+str(i+1) for i in range(m)] + ['A'+str(i+1) for i in range(n)] # Stores the name of the variables
  basic_Var = var[len(var)-n : ]

  dic = {i: 0 for i in var}

  while True: 
    minRatio = []
    zj_cj = []
    for i in range(m+n):
      zj_cj.append(Zj(A, i, b) - c[i])
    
    if all(x >=0 for x in zj_cj):
      gv = 0
      for i in var[:m]:
        gv += dic[i]
      gv = 1 / gv
      zj_cj = [i*gv for i in zj_cj]
      b = [i*gv for i in b]

      a = []
      for i, j in zip(var[m:], zj_cj[m:]):
        a.append(round(j, 2))
      print('Optimal strategy of player A is ',a)

      b = []
      for i in var[:m]:
        b.append(round(dic[i]*gv, 2))
      print('Optimal strategy of player B is ',b)  
     
      gv = gv - k   
      print('\nValue Of game: ', round(gv,2))
      break
    
    #Find the key column 
    min_zj, pos_zj = Min(zj_cj, m_type = 'zj_cj')
    
    #Calculate min ratio
    for i in range(len(A)):
      minRatio.append((x[i] / A[i][pos_zj]) if A[i][pos_zj] != 0 else 0)
    
    #Find the key row
    min_r, pos_r = Min(minRatio)

    if min_r == np.inf: #check for unbounded region
      print('Unbounded')
      break

    pivot = A[pos_r][pos_zj]

    A[pos_r] = [i/pivot for i in A[pos_r]]
    x[pos_r] = x[pos_r]/pivot

    for i in range(n):
      if i != pos_r:
        fact = A[i][pos_zj]
        A[i] = [A[i][j] - fact*A[pos_r][j]
                for j in range(len(var))]
        x[i] = x[i] - fact*x[pos_r]

    #change leaving variable with entering variable
    basic_Var[pos_r] = var[pos_zj]
    b[pos_r] = c[pos_zj]

    for i in dic.keys():
      if i in basic_Var:
        dic[i] = x[basic_Var.index(i)]
      else:
        dic[i] = 0

In [15]:
Value_of_game(A,n,m)

Optimal strategy of player A is  [0.46, 0.0, 0.54]
Optimal strategy of player B is  [0.62, 0.38, 0.0]

Value Of game:  0.31
